In [ ]:
# Import necessary libraries
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from raphtory import Graph

class ContactSequenceDiagram:
    def __init__(self, graph, earliest_time=None, latest_time=None, figsize=(12, 5), save_file=False, filename='Contact_Sequence.PNG'):
        self.graph = graph
        # Control file saving
        self.save_file = save_file
        # Set earliest_time and latest_time from graph if not provided
        if earliest_time is None:
            self.earliest_time = graph.earliest_time
        else:
            self.earliest_time = earliest_time
        if latest_time is None:
            self.latest_time = graph.latest_time
        else:
            self.latest_time = latest_time
        # Set a size of the diagram
        self.figsize = figsize
        # Name of the file, the user may change name and format of the file
        self.filename = filename
        # Choose a colour for a node using custom_palette
        self.node_colours = self.generate_colours()
        # Get indexes of nodes in graph
        self.y_positions = {node.name: i for i, node in enumerate(graph.vertices)}

    def generate_colours(self):
        """ 
        Generate random distinct colors for each node using custom_palette.
        """
        custom_palette = ['#e6194b', '#3cb44b', '#ffe119', '#4363d8', '#f58231',
                          '#911eb4', '#46f0f0', '#f032e6', '#bcf60c', '#fabebe', 
                          '#ea476f', '#2f8d3b', '#fee74c', '#2748c0', '#f79f61',
                          '#b129db', '#75f3f3', '#f361eb', '#97c707', '#f68e8e']
        return {node.name: custom_palette[i % len(custom_palette)] for i, node in enumerate(self.graph.vertices)}

    def set_custom_x_ticks(self, max_value, interval=10):
        # Generate a list of tick positions, which is every integer from 0 to max_value
        all_ticks = list(range(max_value + 1))
        # Generate a list of labels, with labels only at the specified interval and empty strings for others
        labels = [str(i) if i % interval == 0 else '' for i in all_ticks]
        # Set the ticks on the x-axis to the all_ticks
        plt.xticks(ticks=all_ticks, labels=labels)
        # Set the x-axis limits to include a bit before the earliest time and a bit after the max_value
        plt.xlim(self.earliest_time - (interval/2.5), max_value + (interval/4))

    def plot_horizontal_line(self, node):
        """
        Plot a horizontal line for the node at the given y_position using the node's colour.
        """
        plt.hlines(self.y_positions[node], xmin=(self.earliest_time - 0.75), xmax=self.latest_time, 
                   colors=self.node_colours[node], linestyles='solid', lw=3.5, alpha=0.4)

    def plot_interaction(self, source, target, timestamps):
        """
        Plot an edge with a dot and a connecting line.
        edge.src and edge.dst are the node labels, time is the x position.
        All lines representing edges are in black colour.
        """
        for time in timestamps:
            # Plot a dot for the source node
            plt.plot(time, self.y_positions[source], 'o', color=self.node_colours[source], markersize=10)
            # Plot a dot for the target if different from the source
            if source != target:
                plt.plot(time, self.y_positions[target], 'o', color=self.node_colours[target], markersize=10)
                # Calculate the arc's lenght/height based on source and target positions
                arc_height = -0.2 if self.y_positions[source] < self.y_positions[target] else 0.2
                # Create a patch for the arc
                arc = mpatches.FancyArrowPatch((time, self.y_positions[source]), (time, self.y_positions[target]), 
                                                   connectionstyle=f"arc3,rad={arc_height}", color='black', lw=1)
                # Plot the arcs
                plt.gca().add_patch(arc)

    def setup_plot(self):
        """
        Initialize the plot with labels, axes, etc.
        """
        # Set a size of the diagram
        plt.figure(figsize=self.figsize)
        # Set a label on the x-axis
        plt.xlabel('Time')
        # Set the title
        plt.title('Schematic Contact Sequence')
        # Get rid of lines on the x-axis points
        plt.grid(False)

    def finalize_plot(self, ):
        """
        Display the plot and if fancy, save it to a file.
        """
        # Set the y-ticks to node labels
        plt.yticks(range(len(self.graph.vertices)), list(self.graph.vertices.name))
        # Call the custom function to set the x-ticks and labels
        self.set_custom_x_ticks(max_value=self.latest_time, interval=10)
        # Set the y-axis limits to include all nodes
        plt.ylim(-1, len(self.graph.vertices))
        plt.tight_layout()
        # Saves the plot as a file only if save_file is True
        if self.save_file:
            plt.savefig(self.filename)
        # Saves the plot as a file only if save_file is True
        if self.save_file:
            try:
                plt.savefig(self.filename)
                print(f"Diagram saved as {self.filename}")
            except Exception as e:
                print(f"Error saving file: {e}")
        # Display the diagram
        plt.show()

    def create_diagram(self, save_file=False):
        # Set up a save_file flag for finalize_plot() function
        if save_file is not False:
            self.save_file = save_file
        self.setup_plot()
        # Iterate over all vertices with their name
        for node in self.graph.vertices:
            self.plot_horizontal_line(node.name)
        # Iterate over edges with their full history of timestamps
        for edge in self.graph.edges:
            self.plot_interaction(edge.src.name, edge.dst.name, edge.history())
        self.finalize_plot()